# Audio MNIST

In [2]:
from setup import setUp
import glob
import os
import pandas as pd
from data_processing import DataProcessing

# Initialize the setup class with the config file path
setup = setUp(cfg_filepath = 'config.yaml')

# Get the source path, meta data path, and destination path
source_path = setup.source_path
meta_data_path = setup.source_meta_path
destination_path = setup.destination_path
meta_data = setup.meta_data

# Define the target sample rate
target_sr = 8000

# Initialize the DataProcessing class with the destination path, and target sample rate
DP = DataProcessing(destination_path, target_sr)

# Create empty dataframe with column names
df = pd.DataFrame(columns=['mean', 'std', 'avg', 'median', 'min', 'max', 'skewness', 'kurtosis', 'gender', 'digit'])

# Loop over the audio recordings in the source path
all_folders = len(next(os.walk(source_path))[1])+1
for i in range(1, 2): # instert all_folders when done testing
    src_temp = os.path.join(source_path, f"{i:02d}")
    filepath_filename = sorted(glob.glob(os.path.join(src_temp, "*.wav")))
    for file in filepath_filename:
        # read audio data
        fs, data = DP.read_audio(file)
        
        # resample audio data
        data = DP.resample_data(fs, data, target_sr)
        
        # zero padding audio data
        data = DP.zero_pad(data, target_sr)

        # FFT audio data
        data = DP.fft_data(data)

        # Feature creation
        features = DP.fft_features(data)

        # Normalize features
        n_features = DP.normalize_features(features)

        # Add gender and digit label
        dig, vp, rep = file.rstrip(".wav").split("/")[-1].split("_")
        features = DP.add_gender(n_features, meta_data[vp]["gender"])
        features = DP.add_digit(n_features, dig[-1])

        # Append new dict values to the DataFrame
        df = df.append(features, ignore_index=True)
        
# Save data to CSV
DP.save_df_to_csv(df, os.path.join(destination_path, "audio_data.csv"))